In [1]:
import time
import sys
import string
import numpy as np
import lda
import gensim

from sklearn import feature_extraction
from sklearn.cross_validation import train_test_split 
from sklearn.metrics import precision_recall_curve
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB  
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn import metrics  

C:\Users\Administrator.SKY-20170617YWV\Anaconda2\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Administrator.SKY-20170617YWV\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def loadClassData(filename):
    dataList  = []
    for line in open('../input/'+filename,'r').readlines():#读取分类序列
        dataList.append(int(line.strip()))
    return dataList

def loadTrainData(filename):
    dataList  = []
    for line in open('../input/'+filename,'r').readlines():#读取分类序列
        dataList.append(line.strip())
    return dataList

In [3]:
trainCorpus = []
classLabel = []

classLabel = loadClassData('classLabel.txt')
trainCorpus = loadTrainData('trainLeft.txt')  
trainData, testData, trainLabel, testLabel = train_test_split(trainCorpus, classLabel, test_size = 0.2,random_state=0) 

In [4]:
#计算F值分数  
def totalScore(pred,y_test):
    A = 0
    C = 0
    B = 0
    D = 0
    for i in range(len(pred)):
        if y_test[i] == 0:
            if pred[i] == 0:
                A += 1
            elif pred[i] == 1:
                B += 1
        elif y_test[i] == 1:
            if pred[i] == 0:
                C += 1
            elif pred[i] == 1:
                D +=1
    rb_pr = 1.0*D/(B+D)
    rb_re = 1.0*D/(C+D)
    rt_pr = 1.0*A/(A+C)
    rt_re = 1.0*A/(A+B)
    
    Frb = 0.65*rb_pr + 0.35*rb_re
    Frt = 0.65*rt_pr + 0.35*rt_re
    Ftotal = 0.7*Frb + 0.3*Frt
    print Ftotal

In [46]:
def logisticReg(trainData,testData,trainLabel,testLabel):

    vectorizer = CountVectorizer(binary=True)#0.971387536109
#     vectorizer =TfidfVectorizer(binary=True) #0.931660369875
    fea_train = vectorizer.fit_transform(trainData)
    fea_test = vectorizer.transform(testData);  
    clf =  LogisticRegression()
    clf.fit(fea_train,np.array(trainLabel)) 
    pred= clf.predict(fea_test)
    totalScore(pred,testLabel)

In [56]:
def withoutFeature(trainData,testData,trainLabel,testLabel):
    vectorizer = CountVectorizer(binary=True) #0.974682247615
#     vectorizer =TfidfVectorizer(binary=True) #0.963440017657
    fea_train = vectorizer.fit_transform(trainData)
    fea_test = vectorizer.transform(testData)
    print type(fea_test)
    print 'Size of fea_train:' + repr(fea_train.shape) 
    print 'Size of fea_test:' + repr(fea_test.shape) 
    print fea_train.nnz
    print fea_test.nnz
    clf = LinearSVC( C= 0.8)
    clf.fit(fea_train,np.array(trainLabel))  
    pred = clf.predict(fea_test) 
    totalScore(pred,testLabel)

In [57]:
#navie bayes classifier
def nbClassifier(trainData,testData,trainLabel,testLabel):
#     vectorizer = CountVectorizer(binary=True) #0.906835307297
    vectorizer =TfidfVectorizer(binary=True) #0.921827974983
    fea_train = vectorizer.fit_transform(trainData)
    fea_test = vectorizer.transform(testData);  
    print 'Size of fea_train:' + repr(fea_train.shape) 
    print 'Size of fea_test:' + repr(fea_test.shape) 
    print fea_train.nnz
    print fea_test.nnz

    clf = MultinomialNB(alpha = 0.01)   
    clf.fit(fea_train,np.array(trainLabel))
    pred = clf.predict(fea_test)
    totalScore(pred,testLabel)

In [67]:
def linearSVCClassifier(trainData,testData,trainLabel,testLabel):
    hv = HashingVectorizer(n_features =80000)
#     vectorizer = make_pipeline(hv,TfidfTransformer()) #0.958681502859
    vectorizer=hv #0.963931013641
    fea_train = vectorizer.fit_transform(trainData)    #return feature vector 'fea_train' [n_samples,n_features]  
    fea_test = vectorizer.transform(testData);  
    print 'Size of fea_train:' + repr(fea_train.shape) 
    print 'Size of fea_train:' + repr(fea_test.shape) 
    print fea_train.nnz
    print fea_test.nnz
    
    clf = LinearSVC( C= 0.8)
    clf.fit(fea_train,np.array(trainLabel))  
    pred = clf.predict(fea_test);  
    totalScore(pred,testLabel)

In [73]:
def ldaClassifier(trainData,testData,trainLabel,testLabel):
    
#     文档-词频矩阵
    vectorizer = CountVectorizer(binary=True)
    fea_train = vectorizer.fit_transform(trainData)
    fea_test = vectorizer.transform(testData);  
    
#     model.topic_word_ 输出主题-词语矩阵
#     model.doc_topic_ 输出文档主题矩阵
    model = lda.LDA(n_topics=20,n_iter= 500,random_state=1)
    doc_topic_train=model.fit_transform(fea_train)  
#     doc_topic_train1=model.doc_topic_
    doc_topic_test = model.transform(fea_test)
    
#     print doc_topic_train.shape
#     print doc_topic_train1.shape
#     print doc_topic_test.shape
    clf = LinearSVC( C= 0.8)
    clf.fit(doc_topic_train,np.array(trainLabel)) 
    pred = clf.predict(doc_topic_test);  
    totalScore(pred,testLabel)

In [75]:
def rfClassifier(trainData,testData,trainLabel,testLabel):
    hv = HashingVectorizer(n_features = 10000,non_negative=True)
    voctorizer = make_pipeline(hv,TfidfTransformer())  
    fea_train = voctorizer.fit_transform(trainData)    #return feature vector 'fea_train' [n_samples,n_features]  
    fea_test = voctorizer.transform(testData);  
    print 'Size of fea_train:' + repr(fea_train.shape) 
    print 'Size of fea_train:' + repr(fea_test.shape) 
    print fea_train.nnz
    print fea_test.nnz
    
    clf = RandomForestClassifier()
    clf.fit(fea_train,np.array(trainLabel))  
    pred = clf.predict(fea_test);  
    totalScore(pred,testLabel)

In [47]:
# CountVectorizer + LogisticRegression
logisticReg(trainData,testData,trainLabel,testLabel)

0.971387536109


In [49]:
# CountVectorizer + LinearSVC
withoutFeature(trainData,testData,trainLabel,testLabel)

<class 'scipy.sparse.csr.csr_matrix'>
Size of fea_train:(40000, 62438)
Size of fea_test:(10000, 62438)
278526
59942
0.963440017657


In [51]:
# TfidfVectorizer + MultinomialNB
nbClassifier(trainData,testData,trainLabel,testLabel)

Size of fea_train:(40000, 62438)
Size of fea_test:(10000, 62438)
278526
59942
0.921827974983


In [68]:
# HashingVectorizer + (TfidfTransformer)+ LinearSVC
linearSVCClassifier(trainData,testData,trainLabel,testLabel)

Size of fea_train:(40000, 80000)
Size of fea_train:(10000, 80000)
278514
69587
0.963931013641


In [74]:
# LDA + LinearSVC
ldaClassifier(trainData,testData,trainLabel,testLabel)

INFO:lda:n_documents: 40000
INFO:lda:vocab_size: 62438
INFO:lda:n_words: 278526
INFO:lda:n_topics: 20
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -4117538
INFO:lda:<10> log likelihood: -3116324
INFO:lda:<20> log likelihood: -3019215
INFO:lda:<30> log likelihood: -2974552
INFO:lda:<40> log likelihood: -2949624
INFO:lda:<50> log likelihood: -2930408
INFO:lda:<60> log likelihood: -2917653
INFO:lda:<70> log likelihood: -2906090
INFO:lda:<80> log likelihood: -2898187
INFO:lda:<90> log likelihood: -2891838
INFO:lda:<100> log likelihood: -2885895
INFO:lda:<110> log likelihood: -2880935
INFO:lda:<120> log likelihood: -2876240
INFO:lda:<130> log likelihood: -2872751
INFO:lda:<140> log likelihood: -2871348
INFO:lda:<150> log likelihood: -2869570
INFO:lda:<160> log likelihood: -2868331
INFO:lda:<170> log likelihood: -2865161
INFO:lda:<180> log likelihood: -2864482
INFO:lda:<190> log likelihood: -2861651
INFO:lda:<200> log likelihood: -2860433
INFO:lda:<210> log likelihood: -2860043
INFO:lda

0.849635829253


In [76]:
# HashingVectorizer + (TfidfTransformer)+ rf
rfClassifier(trainData,testData,trainLabel,testLabel)

Size of fea_train:(40000, 10000)
Size of fea_train:(10000, 10000)
278363
69546
0.928409779329
